In [1]:
!pip install torch


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install transformers


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os 
import json 
import numpy as np 
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
import pandas as pd
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertConfig
import os
import copy
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertConfig
import argparse
from sklearn.metrics import f1_score
from torch import cuda

In [4]:
!pip install pytorch-pretrained-bert


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
class text_dataset(Dataset):
    def __init__(self, x_y_list, vocab_path, max_seq_length=256, vocab = 'base-cased', transform=None):
        self.max_seq_length = max_seq_length
        self.x_y_list = x_y_list
        self.vocab = vocab
        if self.vocab == 'base-cased':
            self.tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False, do_basic_tokenize=True)
        elif self.vocab == 'finance-cased':
            self.tokenizer = BertTokenizer(vocab_file = vocab_path, do_lower_case = False, do_basic_tokenize = True)
        elif self.vocab == 'base-uncased':
            self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True, do_basic_tokenize=True) 
        elif self.vocab == 'finance-uncased':
            self.tokenizer = BertTokenizer(vocab_file = vocab_path, do_lower_case = True, do_basic_tokenize = True)
    
    def __getitem__(self,index):
        tokenized_review = self.tokenizer.tokenize(self.x_y_list[0][index])
        
        if len(tokenized_review) > self.max_seq_length:
            tokenized_review = tokenized_review[:self.max_seq_length]
            
        ids_review  = self.tokenizer.convert_tokens_to_ids(tokenized_review)
        
        mask_input = [1]*len(ids_review)
        
        padding = [0] * (self.max_seq_length - len(ids_review))
        ids_review += padding
        mask_input += padding
        
        input_type = [0]*self.max_seq_length  
        
        assert len(ids_review) == self.max_seq_length
        assert len(mask_input) == self.max_seq_length
        assert len(input_type) == self.max_seq_length 
        
        ids_review = torch.tensor(ids_review)
        mask_input =  torch.tensor(mask_input)
        input_type = torch.tensor(input_type)
        
        sentiment = self.x_y_list[1][index] 
        list_of_labels = [torch.from_numpy(np.array(sentiment))]
        
        input_feature = {"token_type_ids": input_type, "attention_mask":mask_input, "input_ids":ids_review}
        
        return input_feature, list_of_labels[0]
    
    def __len__(self):
        return len(self.x_y_list[0])

In [6]:
def transform_labels(x_y_list):
    dict_labels = {'neutral': 0, 'positive':1, 'negative':1}
    x_y_list_transformed = [[item[0], dict_labels[item[1]]] for item in x_y_list]
    X = np.asarray([item[0] for item in x_y_list_transformed])
    y = np.asarray([item[1] for item in x_y_list_transformed])
    return X, y

In [7]:
def financialPhraseBankDataset(dir_):
    fb_path = os.path.join(dir_, 'FinancialPhraseBank-v1.0')
    data_50 = os.path.join(fb_path, 'Sentences_50Agree.txt')
    sent_50 = []
    rand_idx = 45
    
    with open(data_50, 'rb') as fi:
        for l in fi:
            l = l.decode('utf-8', 'replace')
            sent_50.append(l.strip())
    
    x_y_list_50 = [sent.split("@") for sent in sent_50]
    x50, y50 = transform_labels(x_y_list_50)
    
    data = [x50, y50]
    X_train, X_test, y_train, y_test = train_test_split(data[0], data[1], test_size=0.1, random_state=rand_idx, stratify=data[1])

    y_train = pd.get_dummies(y_train).values.tolist()
    y_test = pd.get_dummies(y_test).values.tolist()
    X_train = X_train.tolist()
    X_test = X_test.tolist()
            
    final_data = [X_train, X_test, y_train, y_test] 
     
    return final_data

In [8]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertConfig

In [9]:
class BertClassification(nn.Module):
   
    def __init__(self, weight_path, num_labels=2, vocab="base-cased"):
        super(BertClassification, self).__init__()
        self.num_labels = num_labels
        self.vocab = vocab 
        if self.vocab == "base-cased":
            self.bert = BertModel.from_pretrained(weight_path)
            self.config = BertConfig(vocab_size_or_config_json_file=28996, hidden_size=768, num_hidden_layers=12, num_attention_heads=12, intermediate_size=3072)

        elif self.vocab == "base-uncased":
            self.bert = BertModel.from_pretrained(weight_path)
            self.config = BertConfig(vocab_size_or_config_json_file=30522, hidden_size=768, num_hidden_layers=12, num_attention_heads=12, intermediate_size=3072)
            
        elif self.vocab == "finance-cased":
            self.bert = BertModel.from_pretrained(weight_path)
            self.config = BertConfig(vocab_size_or_config_json_file=28573, hidden_size=768, num_hidden_layers=12, num_attention_heads=12, intermediate_size=3072)

        elif self.vocab =="finance-uncased":
            self.bert = BertModel.from_pretrained(weight_path)
            self.config = BertConfig(vocab_size_or_config_json_file=30873, hidden_size=768, num_hidden_layers=12, num_attention_heads=12, intermediate_size=3072)

        self.dropout = nn.Dropout(self.config.hidden_dropout_prob)
        self.classifier = nn.Linear(self.config.hidden_size, num_labels)
        nn.init.xavier_normal(self.classifier.weight)
    
    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None, graphEmbeddings=None):
        _, pooled_output = self.bert(input_ids, token_type_ids, attention_mask)
        pooled_output = self.dropout(pooled_output)
       
        logits = self.classifier(pooled_output)
            
        return logits

In [10]:
class dense_opt():
    def __init__(self, model):
        super(dense_opt, self).__init__()
        self.lrlast = .001
        self.lrmain = .00001
        self.optim = optim.Adam(
        [ {"params":model.bert.parameters(),"lr": self.lrmain},
          {"params":model.classifier.parameters(), "lr": self.lrlast},
       ])
    
    def get_optim(self):
        return self.optim

In [11]:
import os
import copy
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertConfig
import argparse
from sklearn.metrics import f1_score

In [12]:
from torch import cuda

In [13]:
labels = {2:'negative', 0:'neutral',1:'positive'}
num_labels= len(labels)

In [14]:
vocab = "finance-uncased"
vocab_path = 'analyst_tone/analyst_tone/vocab'
pretrained_weights_path = "analyst_tone/analyst_tone/pretrained_weights" # this is pre-trained FinBERT weights
fine_tuned_weight_path = "analyst_tone/analyst_tone/fine_tuned.pth"      # this is fine-tuned FinBERT weights
max_seq_length=256
#device='cuda:0'
device='cuda' if cuda.is_available() else 'cpu'

In [15]:
model = BertClassification(weight_path= pretrained_weights_path, num_labels=num_labels, vocab=vocab)

C:\Users\g-lim\AppData\Local\Temp\ipykernel_13216\709340429.py:25: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(self.classifier.weight)


In [16]:
#model.load_state_dict(torch.load(fine_tuned_weight_path, map_location='cuda:0'))
model.load_state_dict(torch.load(fine_tuned_weight_path, map_location=torch.device('cpu')))
model.to(device)

BertClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30873, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )


In [17]:
tokenizer = BertTokenizer(vocab_file = vocab_path, do_lower_case = True, do_basic_tokenize = True)

In [18]:
sentences = ["there is a shortage of capital, and we need extra financing", 
             "growth is strong and we have plenty of liquidity", 
             "there are doubts about our finances", 
             "profits are flat"]

In [19]:
from torch.utils.data import DataLoader
train_loader = DataLoader(sentences, batch_size=1, shuffle=False)

In [20]:
model.eval()
for batch in train_loader:
    for sent in batch: 
        tokenized_sent = tokenizer.tokenize(sent)
        if len(tokenized_sent) > max_seq_length:
            tokenized_sent = tokenized_sent[:max_seq_length]
    
        ids_review  = tokenizer.convert_tokens_to_ids(tokenized_sent)
        mask_input = [1]*len(ids_review)        
        padding = [0] * (max_seq_length - len(ids_review))
        ids_review += padding
        mask_input += padding
        input_type = [0]*max_seq_length
    
        input_ids = torch.tensor(ids_review).to(device).reshape(-1, max_seq_length)
        attention_mask =  torch.tensor(mask_input).to(device).reshape(-1, max_seq_length)
        token_type_ids = torch.tensor(input_type).to(device).reshape(-1, max_seq_length)
    
        with torch.set_grad_enabled(False):
            outputs = model(input_ids, token_type_ids, attention_mask)
            outputs = F.softmax(outputs,dim=1)
       
            print(sent, '\nFinBERT predicted sentiment: ', labels[torch.argmax(outputs).item()], '\n')

there is a shortage of capital, and we need extra financing 
FinBERT predicted sentiment:  negative 

growth is strong and we have plenty of liquidity 
FinBERT predicted sentiment:  positive 

there are doubts about our finances 
FinBERT predicted sentiment:  negative 

profits are flat 
FinBERT predicted sentiment:  neutral 



In [21]:
torch.argmax(outputs)

tensor(0)

In [22]:
tweet = pd.read_csv('USmoneytary.csv')
tweet.head()

,Date,User,twee
0,12/16/2013,mchoicebanyule,australias monetary policy setting has not bee...
1,12/16/2013,faezahideris,fiscal policy monetary policy and growth policy
2,12/16/2013,MohrGarrett,understanding the money supply and monetary po...
3,12/16/2013,tinybiscuitjr,images of sugar plum fairies and monetary poli...
4,12/16/2013,anish_sinha,wpi data rbis mid quarter monetary policy revi...


In [ ]:
# tweet1 = pd.read_csv('TweetData1.csv')
# tweet1.head()

In [ ]:
# frames = [tweet1, tweet]
# tweets = pd.concat(frames)
# tweets

In [23]:
headlines = list(tweet['twee'])

In [24]:
def chunker(seq, batch_size=1000):
    return (seq[pos:pos + batch_size] for pos in range(0, len(seq), batch_size))

In [25]:
max_seq_length=256

In [26]:
import time
start_time = time.time()

for sentence_batch in chunker(list(tweet['twee'])):
    for sent in sentence_batch:
        tokenized_sentences = tokenizer.tokenize(sent)
        if len(tokenized_sentences) > max_seq_length:
            tokenized_sentences = tokenized_sentences[:max_seq_length]
    
        ids_review  = tokenizer.convert_tokens_to_ids(tokenized_sentences)
        mask_input = [1]*len(ids_review)        
        padding = [0] * (max_seq_length - len(ids_review))
        ids_review += padding
        mask_input += padding
        input_type = [0]*max_seq_length
    
        input_ids = torch.tensor(ids_review).to(device).reshape(-1, max_seq_length)
        attention_mask =  torch.tensor(mask_input).to(device).reshape(-1, max_seq_length)
        token_type_ids = torch.tensor(input_type).to(device).reshape(-1, max_seq_length)
    
        with torch.set_grad_enabled(False):
            outputs = model(input_ids, token_type_ids, attention_mask)
            outputs = F.softmax(outputs,dim=1)
            tweet.loc[tweet['twee'] == sent, 'sentiment'] = labels[torch.argmax(outputs).item()]
    print("--- %s seconds ---" % (time.time() - start_time))

--- 579.1448094844818 seconds ---
--- 1203.2038407325745 seconds ---
--- 1839.0341725349426 seconds ---
--- 2453.4733576774597 seconds ---
--- 3066.9797444343567 seconds ---
--- 3691.8646783828735 seconds ---
--- 4302.720986127853 seconds ---
--- 4912.917807817459 seconds ---
--- 5528.8486161231995 seconds ---
--- 6151.324815273285 seconds ---
--- 6762.093503236771 seconds ---
--- 7372.5984761714935 seconds ---
--- 7982.934673309326 seconds ---
--- 8591.759633302689 seconds ---
--- 9206.238329410553 seconds ---
--- 9816.925726413727 seconds ---
--- 10424.156040668488 seconds ---
--- 11039.217862129211 seconds ---
--- 11647.294757604599 seconds ---
--- 12254.151119470596 seconds ---
--- 12864.356486082077 seconds ---
--- 13474.437756061554 seconds ---
--- 14084.383423089981 seconds ---
--- 14693.219878435135 seconds ---
--- 15301.923071146011 seconds ---
--- 15924.570262908936 seconds ---
--- 16537.458084583282 seconds ---
--- 17152.103796482086 seconds ---
--- 17768.49943780899 seconds

TypeError: 'float' object is not iterable

In [ ]:
tweet

In [ ]:
tweet.to_csv('USmonetary_finBert.csv')

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np

In [ ]:
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

In [ ]:
labels = {0:'neutral', 1:'positive',2:'negative'}

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
tweet = pd.read_csv('Test.csv')
tweet.head()

In [ ]:
headlines = list(tweet['twee'])

In [ ]:
max_length = 256

In [ ]:
headlines_encoding = tokenizer(headlines, truncation=True, padding=True)

In [ ]:
import torch

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        #item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [ ]:
Dataset = Dataset(headlines_encoding, labels)

In [ ]:
Dataset

In [ ]:
model.to(device)

In [ ]:
from torch.utils.data import DataLoader
Loader = DataLoader(Dataset, batch_size=16, shuffle=False)

In [ ]:
Loader

In [ ]:
from transformers import AdamW
optim = AdamW(model.parameters(), lr=5e-5)

In [ ]:
import time
start_time = time.time()

model.eval()
for batch in Loader:
    optim.zero_grad()
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    #labels = batch['labels'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask)
    #tweet.loc[tweet['twee'] == sent, 'sentiment'] = labels[torch.argmax(outputs).item()]
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
outputs